## 🔐 Prerequisites

Before running the first cell, make sure you're authenticated with Azure CLI. Run this command in your terminal:

```bash
az login
```

or

```bash
az login --use-device-code
```

# 🔌 Azure AI Agent with Hosted MCP Tools

This notebook demonstrates integration of Azure AI Agents with hosted Model Context Protocol (MCP) servers, including user approval workflows for function call security.

## Features Covered:
- Setting up Azure AI Agents with Hosted MCP tools
- Using `HostedMCPTool` to connect to external MCP servers
- Implementing user approval workflows for secure function calls
- Thread-based conversation management
- Querying Microsoft Learn documentation via MCP

### ⚠️ Important Note ⚠️
> **MCP (Model Context Protocol) allows agents to access external tools and services. User approval workflows ensure secure function execution.**

## Prerequisites

Before running this notebook, ensure you have:

1. **Azure AI Project**: Access to an Microsoft Foundry project with deployed models
2. **Authentication**: Azure CLI installed and authenticated (`az login --use-device-code`)
3. **Environment Variables**: Set up your `.env` file with:
   - `AI_FOUNDRY_PROJECT_ENDPOINT`
   - `AZURE_AI_MODEL_DEPLOYMENT_NAME`
4. **Dependencies**: Required agent-framework packages installed

If you need to use a different tenant:
```bash
az login --tenant <tenant-id>
```

## Import Libraries

Import the required libraries using the `AzureAIProjectAgentProvider` API:

In [ ]:
import os
from pathlib import Path
from typing import Any

from agent_framework import AgentProtocol, AgentResponse, AgentThread, ChatMessage, HostedMCPTool
from agent_framework.azure import AzureAIProjectAgentProvider
from azure.identity.aio import AzureCliCredential
from dotenv import load_dotenv

# Load environment variables
notebook_path = Path().absolute()
load_dotenv('../../.env')

# Verify environment setup
endpoint = os.getenv('AI_FOUNDRY_PROJECT_ENDPOINT')
model = os.getenv('AZURE_AI_MODEL_DEPLOYMENT_NAME')

print("🔧 Environment Configuration:")
print(f"✅ Project Endpoint: {endpoint[:50]}..." if endpoint else "❌ AI_FOUNDRY_PROJECT_ENDPOINT not set")
print(f"✅ Model Deployment: {model}" if model else "❌ AZURE_AI_MODEL_DEPLOYMENT_NAME not set")

## Define User Approval Handler 🔐

The approval handler implements a security workflow that requires user confirmation before executing MCP tool calls. This is important for:
- **Security**: Prevents unauthorized function execution
- **Transparency**: Users can see what actions the agent wants to perform
- **Control**: Users maintain control over sensitive operations

In [ ]:
async def handle_approvals_without_thread(query: str, agent: "AgentProtocol") -> AgentResponse:
    """When we don't have a thread, we need to ensure we return with the input,
    approval request and approval."""
    result = await agent.run(query, store=False)
    
    while len(result.user_input_requests) > 0:
        new_inputs: list[Any] = [query]
        
        for user_input_needed in result.user_input_requests:
            print(
                f"🔐 User Input Request for function from {agent.name}: "
                f"{user_input_needed.function_call.name}"
                f" with arguments: {user_input_needed.function_call.arguments}"
            )
            new_inputs.append(ChatMessage(role="assistant", contents=[user_input_needed]))
            user_approval = input("Approve function call? (y/n): ")
            new_inputs.append(
                ChatMessage(role="user", contents=[user_input_needed.create_response(user_approval.lower() == "y")])
            )
        
        result = await agent.run(new_inputs, store=False)
    
    return result


async def handle_approvals_with_thread(
    query: str, 
    agent: "AgentProtocol", 
    thread: "AgentThread"
) -> AgentResponse:
    """Here we let the thread deal with the previous responses, and we just rerun with the approval."""
    result = await agent.run(query, thread=thread)
    
    while len(result.user_input_requests) > 0:
        new_input: list[Any] = []
        
        for user_input_needed in result.user_input_requests:
            print(
                f"🔐 User Input Request for function from {agent.name}: "
                f"{user_input_needed.function_call.name}"
                f" with arguments: {user_input_needed.function_call.arguments}"
            )
            user_approval = input("Approve function call? (y/n): ")
            new_input.append(
                ChatMessage(
                    role="user",
                    contents=[user_input_needed.create_response(user_approval.lower() == "y")],
                )
            )
        
        result = await agent.run(new_input, thread=thread)
    
    return result

## Create Agent with Hosted MCP Tool 🔌

The `HostedMCPTool` connects to external MCP servers that provide additional capabilities. In this example, we connect to Microsoft Learn's MCP endpoint to answer documentation questions.

**Key Components:**
- `HostedMCPTool`: Configures the connection to an MCP server
- `name`: A friendly name for the tool
- `url`: The MCP server endpoint URL
- `approval_mode`: Controls when user approval is required (`"never_require"` or `"always_require"`)

In [ ]:
async def run_hosted_mcp_without_approval() -> None:
    """Example showing MCP Tools without approval."""
    print("=== 🔌 MCP without approvals ===")
    
    async with (
        AzureCliCredential() as credential,
        AzureAIProjectAgentProvider(credential=credential) as provider,
    ):
        # Create agent with Hosted MCP Tool - no approval required
        agent = await provider.create_agent(
            name="my-learn-docs-agent",
            instructions="You are a helpful assistant that can help with Microsoft documentation questions.",
            tools=HostedMCPTool(
                name="Microsoft Learn MCP",
                url="https://learn.microsoft.com/api/mcp",
                approval_mode="never_require",
            ),
        )
        
        print(f"✅ Created agent: {agent.name}")
        print(f"🔌 MCP Tool: Microsoft Learn MCP (approval_mode=never_require)\n")
        
        query = "How to create an Azure storage account using az cli?"
        print(f"🤔 User: {query}")
        result = await agent.run(query)
        print(f"📚 {agent.name}: {result}\n")

## Execute Without Approval 🚀

Run the example without approval mode - the agent will execute MCP tool calls automatically:

In [ ]:
# Run without approval mode
await run_hosted_mcp_without_approval()

## MCP with Approval Mode 🔐

For scenarios where you want user approval for function calls (recommended for security):

> ⚠️ **Note:** The approval workflow with `approval_mode="always_require"` requires interactive user input which does not work reliably in Jupyter notebooks due to the asynchronous nature of the execution. 
>


In [ ]:
async def run_hosted_mcp_with_approval() -> None:
    """Example showing MCP Tools with approvals (without thread)."""
    print("=== 🔐 MCP with approvals ===")
    
    async with (
        AzureCliCredential() as credential,
        AzureAIProjectAgentProvider(credential=credential) as provider,
    ):
        # Create agent with Hosted MCP Tool - always require approval
        agent = await provider.create_agent(
            name="my-api-specs-agent",
            instructions="You are a helpful agent that can use MCP tools to assist users.",
            tools=HostedMCPTool(
                name="api-specs",
                url="https://gitmcp.io/Azure/azure-rest-api-specs",
                approval_mode="always_require",
            ),
        )
        
        print(f"✅ Created agent: {agent.name}")
        print(f"🔌 MCP Tool: api-specs (approval_mode=always_require)\n")
        
        query = "Please summarize the Azure REST API specifications Readme"
        print(f"🤔 User: {query}")
        result = await handle_approvals_without_thread(query, agent)
        print(f"📚 {agent.name}: {result}\n")

# ⚠️ This will NOT work in notebook - run as a script instead
# await run_hosted_mcp_with_approval()

## Key Takeaways 📚

### Hosted MCP Tools

```python
from agent_framework import HostedMCPTool
from agent_framework.azure import AzureAIProjectAgentProvider

agent = await provider.create_agent(
    name="my-mcp-agent",
    instructions="...",
    tools=HostedMCPTool(
        name="Tool Name",
        url="https://example.com/api/mcp",
        approval_mode="never_require",  # or "always_require"
    ),
)
```

### User Approval Workflow

1. **Request Detection**: Check `result.user_input_requests` for pending approvals
2. **User Prompt**: Display function name and arguments to user
3. **Approval Response**: Use `user_input_needed.create_response(True/False)`
4. **Re-run Agent**: Continue execution with approval responses

### Benefits of MCP Integration

| Feature | Benefit |
|---------|----------|
| External Services | Access documentation, APIs, and tools |
| Security | User approval workflow for sensitive operations |
| Flexibility | Connect to any MCP-compatible server |
| Context | Thread-based conversation management |

### Approval Modes

| Mode | Description |
|------|-------------|
| `"never_require"` | Tool calls execute automatically without user approval |
| `"always_require"` | Every tool call requires explicit user approval |

### Available MCP Endpoints

- **Microsoft Learn**: `https://learn.microsoft.com/api/mcp` - Documentation search and retrieval
- **Azure REST API Specs**: `https://gitmcp.io/Azure/azure-rest-api-specs` - Azure API specifications
- Custom MCP servers can be hosted for your specific needs

### Best Practices

1. **Security First**: Always implement approval workflows for sensitive operations
2. **Thread Management**: Use threads for multi-turn conversations
3. **Error Handling**: Handle network failures and approval denials gracefully
4. **Logging**: Log all function calls and approvals for audit purposes
5. **Agent Names**: Use hyphens (not underscores) in agent names

⚠️ **Security Note**: Use `approval_mode="never_require"` only in trusted environments. For production, prefer `approval_mode="always_require"`.